In [3]:
# from openai import OpenAI
import toml
# from ollama import chat
# from ollama import ChatResponse
from ollama import Client
from langchain_core.prompts import PromptTemplate
import json
from langchain_ollama.llms import OllamaLLM
from langchain.chains.llm import LLMChain


In [2]:

# Load a TOML file
with open('../secrets.toml', 'r') as f:
    openai_key = toml.load(f).get('api_keys').get('openai')

assert openai_key, "OpenAI API key not found in secrets.toml"


In [3]:
class LLMModel:
    def __init__(self, model_name: str, run_local: bool = True, system_prompt: str = "you are a helpful assistant"):
        self.run_local = run_local
        self.model_name = model_name
        self.system_prompt = system_prompt
        self.messages = [{"role": "system", "content": self.system_prompt}]
        
        if self.run_local:
            self.client = Client(
                host='http://localhost:11434',
                # headers={'x-some-header': 'some-value'}
                )

    def add_message(self, role: str, content: str) -> bool:
        if role not in ["system", "user", "assistant"]:
            raise ValueError("Role must be 'system', 'user', or 'assistant'")
        self.messages.append({"role": role, "content": content})
        return True

    def clear_messages(self) -> bool:
        self.messages = []
        return True

    def local_generate(self, prompt: str, max_tokens: int = 1500, **kwargs) -> str:
        self.messages.append({"role": "user", "content": prompt})
        response = self.client.chat(model=self.model_name, messages=self.messages, **kwargs)
        self.messages.append({"role": "assistant", "content": response.message.content.strip()})
        return response.message.content.strip()
    
    # question: how do I set max token in ollama python api?
    


In [4]:
model = LLMModel(model_name="llama3.2:latest", run_local=True)

In [5]:
# model.local_generate("What is the capital of India?")

In [6]:
# model.local_generate("What is the weather like there?")

In [7]:
# model.local_generate("What is the best month to visit?")

In [8]:
# model.local_generate("What is the best month to visit? Just give me the month name, no other text.")

In [9]:
# model.messages

In [ ]:
# model.clear_messages()

True

In [35]:
topic = "The Emergency in India (1975-1977)"
number_of_subtopics = 5


In [ ]:
subtopics_template = PromptTemplate.from_template(
    """Generate a list of {number_of_subtopics} subheadings for a highschool essay on {topic}.
    Order the subheadings such that they have a logical sequence.
    Each subheading should have a one-line discription.
    Provide output in json format as follows:

    {{<heading1>:<description1>,
    <heading2>:<description2>,...}}"""
)

In [ ]:
subtopics_prompt = subtopics_template.format(topic=topic, number_of_subtopics=number_of_subtopics)
subtopics_prompt

'Generate a list of 5 subheadings for a highschool essay on The Emergency in India (1975-1977).\n    Order the subheadings such that they have a logical sequence.\n    Each subheading should have a one-line discription.\n    Provide output in json format as follows:\n\n    {<heading1>:<description1>,\n    <heading2>:<description2>,...}'

In [47]:
topics = model.local_generate(subtopics_prompt, format='json')
topics = json.loads(topics)
topics

{'Causes_of_the_emergency': 'Analyze the historical events that led to the declaration of emergency by Prime Minister Indira Gandhi in 1975.',
 'Effects_on_indian_politics': "Explain how the Emergency affected India's political landscape and its impact on the country's democratic institutions.",
 'Censorship_and_media_control': 'Discuss the measures taken by the government to control media coverage during the Emergency period, including censorship and suppression of opposition voices.',
 'Dissolution_of_constituent_assemblies': 'Describe the dissolution of the constituent assemblies at the Centre and the States by the President on June 25, 1975, marking a significant turning point in the Emergency',
 'Legacy_of_the_emergency_and_reforms': "Assess the lasting impact of the Emergency period on India's politics, economy, and society, including the subsequent reforms introduced by Prime Minister Rajiv Gandhi."}

In [48]:
for k,v in topics.items():
    print(k)
    print(v)

Causes_of_the_emergency
Analyze the historical events that led to the declaration of emergency by Prime Minister Indira Gandhi in 1975.
Effects_on_indian_politics
Explain how the Emergency affected India's political landscape and its impact on the country's democratic institutions.
Censorship_and_media_control
Discuss the measures taken by the government to control media coverage during the Emergency period, including censorship and suppression of opposition voices.
Dissolution_of_constituent_assemblies
Describe the dissolution of the constituent assemblies at the Centre and the States by the President on June 25, 1975, marking a significant turning point in the Emergency
Legacy_of_the_emergency_and_reforms
Assess the lasting impact of the Emergency period on India's politics, economy, and society, including the subsequent reforms introduced by Prime Minister Rajiv Gandhi.


In [50]:
def generate_subtopics(topic: str, number_of_subtopics: int, model: LLMModel) -> dict:
    subtopics_template = PromptTemplate.from_template(
    """Generate a list of {number_of_subtopics} subheadings for a highschool essay on {topic}.
    Order the subheadings such that they have a logical sequence.
    Each subheading should have a one-line discription.
    Provide output in json format as follows:

    {{<heading1>:<description1>,
    <heading2>:<description2>,...}}"""
    )
    
    subtopics_prompt = subtopics_template.format(topic=topic, number_of_subtopics=number_of_subtopics)

    topics = model.local_generate(subtopics_prompt, format='json')
    topics = json.loads(topics)
    return topics

In [27]:
def subtopic_chain() -> LLMChain:
    subtopics_template = PromptTemplate.from_template(
        """Generate a list of {number_of_subtopics} subheadings for a highschool essay on {topic}.
        Order the subheadings such that they have a logical sequence.
        Each subheading should have a one-line discription.
        Provide output in json format as follows:

        {{<heading1>:<description1>,
        <heading2>:<description2>,...}}
        
        Only return the json object without any additional text."""
    )
    
    model = OllamaLLM(model="llama3.2:latest", run_local=True)

    return LLMChain(llm=model, prompt=subtopics_template)

In [51]:
generate_subtopics("History of the Internet", 6, model)

{'Early Beginnings': 'Discuss the origins of the internet, from its development in the 1960s by the United States Department of Defense to its first functional networks.',
 'ARPANET and Network Expansion': 'Explain how ARPANET, the first operational packet switching network, expanded into a global network through various government and private initiatives.',
 'TCP/IP and the Internet Protocol': 'Describe the development of TCP/IP (Transmission Control Protocol/Internet Protocol), which became the standard communication protocol for the internet.',
 'World Wide Web and HTML': 'Introduce the World Wide Web and Hypertext Markup Language (HTML), technologies invented by Tim Berners-Lee that enabled web browsing and online content sharing.',
 'Internet Governance and Regulation': 'Analyze the evolution of internet governance, including the establishment of organizations such as ICANN (Internet Corporation for Assigned Names and Numbers) and efforts to regulate online content.',
 'Impact and

In [52]:
generate_subtopics("Evolution of Agentic AI", 10, model)

{'Introduction to Agentic AI': 'Introduce the concept of agentic AI, which refers to artificial intelligence that can make decisions and act independently.',
 'Early Foundations: Robotics and Simulation': 'Discuss the early foundations of agentic AI, including robotics and simulation research that laid the groundwork for more advanced systems.',
 'Machine Learning and Reinforcement Learning': 'Explain how machine learning and reinforcement learning techniques have been used to create agentic AI systems that can learn from experience and make decisions.',
 'Development of AlphaGo: A Key Breakthrough': 'Describe the development of AlphaGo, a computer program that defeated a human world champion in Go, marking a significant milestone in agentic AI research.',
 'Modern Agentic AI: Applications and Use Cases': 'Discuss various applications and use cases of modern agentic AI, including autonomous vehicles, drones, and smart home systems.',
 'Value Alignment and Ethics': 'Analyze the importan

In [32]:
chain = subtopic_chain()
subtopics = chain.run(topic="History of the Internet", number_of_subtopics=6)
subtopics = json.loads(subtopics)
subtopics

{'Early Beginnings': 'The origins of the internet date back to the 1960s, with the development of ARPANET by the US Department of Defense.',
 'Network Expansion and TCP/IP': 'The expansion of the internet led to the creation of the Transmission Control Protocol/Internet Protocol (TCP/IP), a standard that enabled communication between different networks.',
 'The Internet Goes Public': 'In the 1980s, the internet was opened to the public, with the introduction of services such as America Online and CompuServe.',
 'World Wide Web and HTTP': 'The World Wide Web was invented by Tim Berners-Lee in 1989, revolutionizing the way people accessed and shared information online.',
 "The Internet's Impact on Society": 'The internet has had a profound impact on modern society, transforming the way we communicate, work, and access information.',
 'Modern Internet and Future Developments': 'Today, the internet continues to evolve, with advancements in technologies such as artificial intelligence, bloc

In [14]:
model = OllamaLLM(model="llama3.2:latest")
model.invoke("Come up with 10 names for a song about parrots")

'Here are 10 potential song title ideas about parrots:\n\n1. "Feathered Friends Forever"\n2. "Parrot\'s Lament"\n3. "Rainbow Wings"\n4. "Squawk Serenade"\n5. "Colorful Chaos"\n6. "The Parrot\'s Prayer"\n7. "Wings of Wonder"\n8. "Mimicry Blues"\n9. "Sunset in the Aviary"\n10. "Flock to Me"\n\nLet me know if you\'d like me to suggest more!'